# Test imputation on untyped SNPs chrom 20

In [1]:
using Revise
using VCFTools
using MendelImpute
using GeneticVariation
using Random
using StatsBase
using CodecZlib
using ProgressMeter
using JLD2, FileIO, JLSO
using BenchmarkTools
using GroupSlices
using TimerOutputs
using LinearAlgebra

BLAS.set_num_threads(1)

┌ Info: Precompiling Revise [295af30f-e4ad-537b-8983-00126c2a3abe]
└ @ Base loading.jl:1278
┌ Info: Precompiling VCFTools [a620830f-fdd7-5ebc-8d26-3621ab35fbfe]
└ @ Base loading.jl:1278
[ Info: Compiling VCF parser...
┌ Info: Precompiling MendelImpute [e47305d1-6a61-5370-bc5d-77554d143183]
└ @ Base loading.jl:1278


# MendelImpute error rate (window-window intersection)

In [2]:
Threads.nthreads()

8

In [3]:
# 8 threads
Random.seed!(2020)
d       = 1000
tgtfile = "target.chr20.typedOnly.maf0.01.masked.vcf.gz"
reffile = "ref.chr20.maxd$d.maf0.01.excludeTarget.jlso"
outfile = "mendel.chr20.imputed.target.vcf.gz"
@time ph = phase(tgtfile, reffile, outfile = outfile, max_d = d,
    dynamic_programming = false);

# import imputed result and compare with true
X_mendel = convert_gt(Float32, outfile)
X_complete = convert_gt(Float32, "target.chr20.full.vcf.gz")
n, p = size(X_mendel)
println("error_rate = ", sum(X_mendel .!= X_complete) / n / p)

Importing reference haplotype data...


Importing genotype file...100%|█████████████████████████| Time: 0:00:28
Computing optimal haplotypes...100%|████████████████████| Time: 0:00:59
Phasing...100%|█████████████████████████████████████████| Time: 0:00:42
Writing to file...100%|█████████████████████████████████| Time: 0:00:10


Total windows = 3252, averaging ~ 510 unique haplotypes per window.

Timings: 
    Data import                     = 63.7058 seconds
        import target data             = 30.7379 seconds
        import compressed haplotypes   = 32.9679 seconds
    Computing haplotype pair        = 60.1253 seconds
        BLAS3 mul! to get M and N      = 0.947467 seconds per thread
        haplopair search               = 50.657 seconds per thread
        initializing missing           = 1.277 seconds per thread
        allocating and viewing         = 0.0796761 seconds per thread
        index conversion               = 0.0750589 seconds per thread
    Phasing by win-win intersection = 42.6274 seconds
        Window-by-window intersection  = 36.3028 seconds per thread
        Breakpoint search              = 0.693719 seconds per thread
        Recording result               = 2.18262 seconds per thread
    Imputation                     = 13.4437 seconds
        Imputing missing               = 2.11